In [1]:
import os

import mala
from mala import printout
from mala.network import TesterGraph

import total_energy as te

In [2]:
parameters = mala.Parameters()

# Currently, the splitting in training, validation and test set are
# done on a "by snapshot" basis.
parameters.data.data_splitting_type = "by_snapshot"
parameters.data.use_graph_data_set = True

# Specify the training parameters.
parameters.running.max_number_epochs = 1
parameters.running.ldos_grid_batch_size = 200
parameters.running.learning_rate = 0.00001
parameters.running.trainingtype = "Adam"
parameters.running.visualisation = 1


parameters.targets.ldos_gridsize = 201
parameters.targets.ldos_gridoffset_ev = -13.5
parameters.targets.ldos_gridspacing_ev = 0.1
parameters.targets.pseudopotential_path = "/bigdata/casus/wdm/Bartek_H2/H128/H.pbe-rrkjus_psl.1.0.0.UPF"

parameters.verbosity = 1

parameters.use_gpu = True

In [3]:
data_handler = mala.DataHandlerGraph(parameters)

# for i in range(16):
for i in range(1):
    data_handler.add_snapshot(
        f'H_snapshot{i}.pw.scf.in', f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}',
        f'H_snapshot{i}.out.npy', '/bigdata/casus/wdm/Bartek_H2/H128/ldos/',
        'tr', calculation_output_file=f'/bigdata/casus/wdm/Bartek_H2/H128/outputs/snapshot{i}.out'
    )
# for i in range(16, 18):
for i in range(1, 2):
    data_handler.add_snapshot(
        f'H_snapshot{i}.pw.scf.in', f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}',
        f'H_snapshot{i}.out.npy', '/bigdata/casus/wdm/Bartek_H2/H128/ldos/',
        'va', calculation_output_file=f'/bigdata/casus/wdm/Bartek_H2/H128/outputs/snapshot{i}.out'
    )

data_handler.prepare_data(reparametrize_scaler=False)

No data rescaling will be performed.
No data rescaling will be performed.
Checking the snapshots and your inputs for consistency.
Consistency check successful.
Data scalers already initilized, loading data to RAM.
Build datasets.
Build dataset: Done.


In [4]:
parameters.network.nn_type = "se3_transformer"
parameters.network.layer_sizes = [
    data_handler.input_dimension,
    16,
    data_handler.output_dimension
]
# Setup network and trainer.
network = mala.Network(parameters)
trainer = mala.TrainerGraph(parameters, network, data_handler)
trainer.train_network()

Writing visualization output to ./mala_logging/2023-07-05-14-00-06
Validating ldos on validation data set.


  0%|          | 0/2430 [00:00<?, ?it/s]/home/brzoza73/casus/mala-venv/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/brzoza73/casus/mala-venv/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
100%|██████████| 2430/2430 [00:43<00:00, 55.43it/s]


Initial Guess - validation data loss:  0.005103628135021822
Validating ldos on validation data set.


100%|██████████| 2430/2430 [00:45<00:00, 53.38it/s]

Epoch 0: validation data loss: 0.001902268256670163
Final validation data loss:  0.001902268256670163


In [5]:
data_handler_test = mala.DataHandlerGraph(parameters)

# for i in range(16):
for i in range(2, 3):
    data_handler.add_snapshot(
        f'H_snapshot{i}.pw.scf.in', f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}',
        f'H_snapshot{i}.out.npy', '/bigdata/casus/wdm/Bartek_H2/H128/ldos/',
        'te', calculation_output_file=f'/bigdata/casus/wdm/Bartek_H2/H128/outputs/snapshot{i}.out'
    )

data_handler_test.prepare_data(reparametrize_scaler=False)


No data rescaling will be performed.
No data rescaling will be performed.
Checking the snapshots and your inputs for consistency.
DataHandler prepared for inference. No training possible with this setup. If this is not what you wanted, please revise the input script. Validation snapshots you may have entered willbe ignored.
Consistency check successful.
Build datasets.
Build dataset: Done.


In [6]:
network = network.to('cuda')
observables_to_test = [
  "ldos",
  "band_energy",
  "band_energy_full",
  "number_of_electrons",
  "total_energy",
  "total_energy_full",
  "density",
  "dos"
]

tester = TesterGraph(parameters, network, data_handler_test, observables_to_test=observables_to_test)
results = tester.test_all_snapshots()

results

100%|██████████| 2430/2430 [00:46<00:00, 51.91it/s]


: 

: 